# The K-Secretary Problem and Optimal Thresholds

The problem setup, definitions, LP formulation, and its interpretation are from Buchbinder Jain Singh '14 and Chan Chen Jiang '15.

## Problem Setup

**Input:**
- $n$ items arrive in uniformly random order
- Items have a total ordering (only relative comparisons observable)
- Algorithm has $K$ quotas for selecting items
- Decisions are **irrevocable** (made upon arrival)

**Goal:** Maximize $\mathbb{E}[\text{number of selected items among the } K \text{ best overall}]$

**Performance ratio:** $\frac{\mathbb{E}[\text{number selected among } K \text{ best}]}{K}$

## Key Definitions

**k-potential:** An item arriving at step $i$ is a **k-potential** if it ranks k-th among all items arrived so far (including itself).

**k≥-potential:** An item is a **k≥-potential** if it is a $k'$-potential for some $k' \leq k$.

**Quotas:** $Q_K, Q_{K-1}, \ldots, Q_1$ (used in order: $Q_K$ first, $Q_1$ last)

## Optimal K-Threshold Algorithm

**Parameters:** $K^2$ thresholds $\{\tau_{j,k} : j \in [K], k \in [K]\}$ satisfying:
1. For each quota $j$: $0 < \tau_{j,1} \leq \tau_{j,2} \leq \cdots \leq \tau_{j,K} \leq 1$
2. For each potential level $k$: $0 < \tau_{K,k} \leq \tau_{K-1,k} \leq \cdots \leq \tau_{1,k} \leq 1$

**Selection rule:** 
- After time $\tau_{j,k}$, quota $Q_j$ may be used for any $k$≥-potential
- Select greedily: whenever an item can be selected, use the highest-indexed available quota

**Intuition:** Each quota "matures" progressively (condition 1), and higher-indexed quotas mature earlier (condition 2).

## Linear Programming Formulation: $\text{LP}_n(K,K)$

### Decision Variables

$$z_{j|k}(i) = \Pr[\text{item at step } i \text{ selected using quota } Q_j \mid \text{it is a } k\text{-potential}]$$

### The Linear Program

$$
\begin{align}
\max \quad & \sum_{j=1}^K \sum_{k=1}^K \sum_{i=1}^n \frac{1}{n} \sum_{\ell=k}^K \delta_{k|\ell}(i) \cdot z_{j|k}(i) \\
\text{s.t.} \quad & z_{j|k}(i) \leq \sum_{m=1}^{i-1} \frac{1}{m} \sum_{\ell=1}^K \left[z_{(j+1)|\ell}(m) - z_{j|\ell}(m)\right], 
\quad \forall i \in [n], k \in [K], 1 \leq j < K \\
& z_{K|k}(i) \leq 1 - \sum_{m=1}^{i-1} \frac{1}{m} \sum_{\ell=1}^K z_{K|\ell}(m), 
\quad \forall i \in [n], k \in [K] \\
& z_{j|k}(i) \geq 0, \quad \forall i \in [n], j, k \in [K] \\
\end{align}
$$

where

$$\delta_{k|\ell}(i) = \frac{\binom{n-i}{\ell-k}\binom{i-1}{k-1}}{\binom{n-1}{\ell-1}}$$

## Interpretation

### The $\delta$ Coefficients

$$\delta_{k|\ell}(i) = \Pr[\text{item at step } i \text{ is a } k\text{-potential} \mid \text{it is the } \ell\text{-th best overall}]$$

**Explanation:** Given the $\ell$-th best item arrives at step $i$, it is a $k$-potential iff exactly $(k-1)$ of the $(\ell-1)$ better items arrive in the $(i-1)$ steps before it.

### Objective Function

Define auxiliary quantities:
- $\gamma_k(i) := \sum_{j=1}^K z_{j|k}(i) = \Pr[\text{item at step } i \text{ selected} \mid \text{it is a } k\text{-potential}]$
- $\beta_\ell(i) := \sum_{k=1}^\ell \delta_{k|\ell}(i) \cdot \gamma_k(i) = \Pr[\ell\text{-th best item selected} \mid \text{it arrives at step } i]$

The objective equals:
$$\sum_{\ell=1}^K \underbrace{\sum_{i=1}^n \frac{1}{n} \beta_\ell(i)}_{\Pr[\ell\text{-th best selected}]}$$

**Interpretation:** Sum over all $K$ best items of the probability each is selected (since each arrives at step $i$ with probability $1/n$).

### Constraints

$$\sum_{m=1}^{i-1} \frac{1}{m} \sum_{\ell=1}^K z_{j|\ell}(m) = \Pr[\text{quota } Q_j \text{ used by step } i]$$

**Explanation:** At each step $m < i$, an $\ell$-potential arrives with probability $1/m$, and quota $Q_j$ is used with probability $z_{j|\ell}(m)$.

The RHS of constraints gives:
- $\sum_{m=1}^{i-1} \frac{1}{m} \sum_{\ell=1}^K [z_{(j+1)|\ell}(m) - z_{j|\ell}(m)] = \Pr[\text{exactly } j \text{ quotas remain at step } i]$ for $j < K$
- $1 - \sum_{m=1}^{i-1} \frac{1}{m} \sum_{\ell=1}^K z_{K|\ell}(m) = \Pr[\text{all } K \text{ quotas remain at step } i]$ for $j = K$

**Constraint meaning:** To select with quota $Q_j$ at step $i$, that quota must be available.

**Theorem:** Any K-secretary algorithm induces feasible $z$ values with objective value equal to $\mathbb{E}[\text{number selected among } K \text{ best}]$.

**Corollary:** The optimal LP value equals the optimal performance ratio achievable by any algorithm.

# Demo: A Framework for Analyzing the Class of Algorithms

In [1]:
import numpy as np
from scipy.special import comb
import gurobipy as gp
from gurobipy import GRB
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [2]:
def compute_delta(n, i, k, ell):
    """
    Compute delta_{k|ell}(i) = P[item at step i is k-potential | it's ell-th best overall]
    """
    if i < 1 or k < 1 or ell < k or k > i:
        return 0.0
    
    numerator = comb(n - i, ell - k, exact=True) * comb(i - 1, k - 1, exact=True)
    denominator = comb(n - 1, ell - 1, exact=True)
    
    return numerator / denominator if denominator > 0 else 0.0

In [3]:
def create_k_secretary_lp_model(n, K, verbose=False):
    """
    Creates a Pyomo ConcreteModel for the base K-secretary LP.
    Does not solve.
    """
    if verbose:
        print(f"Building Pyomo model components for n={n}, K={K}...")

    # Precompute delta
    delta = np.zeros((n, K, K))
    for i in range(1, n + 1):
        for k in range(1, K + 1):
            for ell in range(k, K + 1):
                delta[i-1, k-1, ell-1] = compute_delta(n, i, k, ell)
    
    model = pyo.ConcreteModel()

    # --- Indices ---
    model.J = pyo.Set(initialize=range(K)) # Quotas
    model.K = pyo.Set(initialize=range(K)) # Potentials
    model.I = pyo.Set(initialize=range(n)) # Steps
    model.VARS = model.J * model.K * model.I

    # --- Variables ---
    # z[j, k, i] (using 0-based indexing)
    model.z = pyo.Var(model.VARS, bounds=(0.0, 1.0))

    # --- Objective Function ---
    c_dict = {}
    for j, k, i in model.VARS:
        c_val = 0.0
        for ell in range(k, K): # ell is 0-indexed k..K-1
            c_val += (1.0 / n) * delta[i, k, ell]
        c_dict[(j,k,i)] = c_val
    
    model.objective = pyo.Objective(
        expr=sum(c_dict[v] * model.z[v] for v in model.VARS),
        sense=pyo.maximize
    )

    # --- Constraints ---
    model.lp_constrs = pyo.ConstraintList()

    # Constraints for j < K (j_idx = 0...K-2)
    for j in range(K - 1):
        for k in range(K):
            for i in range(1, n): # i_idx = 1...n-1
                rhs = 0
                for m in range(i): # m_idx = 0...i-1
                    for ell in range(K):
                        rhs += (1.0 / (m + 1)) * (model.z[j+1, ell, m] - model.z[j, ell, m])
                model.lp_constrs.add(model.z[j, k, i] <= rhs)
    
    # Constraints for j = K (j_idx = K-1)
    j = K - 1
    for k in range(K):
        for i in range(1, n): # i_idx = 1...n-1
            rhs = 1.0
            for m in range(i): # m_idx = 0...i-1
                for ell in range(K):
                    rhs -= (1.0 / (m + 1)) * model.z[j, ell, m]
            model.lp_constrs.add(model.z[j, k, i] <= rhs)

    if verbose:
        print("Base model components built.")
        
    return model

In [4]:
def solve_base_lp_pyomo(n, K, verbose=True):
    """
    Solves the base K-secretary LP (B=0) using Pyomo.
    """
    # Create the base model
    model = create_k_secretary_lp_model(n, K, verbose=verbose)

    if verbose:
        print("Base model created. Solving with Gurobi...")

    # Solve
    opt = SolverFactory('gurobi')
    results = opt.solve(model, tee=False) # tee=True to see solver logs
    
    obj_val = 0.0
    if results.solver.termination_condition == pyo.TerminationCondition.optimal:
        obj_val = pyo.value(model.objective)
        if verbose:
            print(f"\n{'='*60}\nOPTIMAL SOLUTION FOUND (B=0)\n{'='*60}")
            print(f"Expected # of top-{K} items selected: {obj_val:.6f}")
            print(f"Competitive ratio: {obj_val / K:.6f}")
            print(f"{'='*60}\n")
    else:
        print(f"Base LP solve failed: {results.solver.termination_condition}")
        return None, None

    return model, obj_val

In [5]:
# Run for n=500, K=2 with Pyomo
n_demo = 100
K_demo = 2

print(f"Solving K-secretary problem with n={n_demo}, K={K_demo} (Pyomo B=0)")
print(f"This is the exact LP from equations (1)-(4)\n")

# We get the solved model and the base objective value
base_model, obj_val_base = solve_base_lp_pyomo(n_demo, K_demo, verbose=True)

Solving K-secretary problem with n=100, K=2 (Pyomo B=0)
This is the exact LP from equations (1)-(4)

Building Pyomo model components for n=100, K=2...
Base model components built.
Base model created. Solving with Gurobi...

OPTIMAL SOLUTION FOUND (B=0)
Expected # of top-2 items selected: 0.987887
Competitive ratio: 0.493943



The LP solution finds a single, fully optimal algorithm. We can, however, analyze the *class* of ordinal algorithms by parameterizing them.

We can formalize this by introducing an external agent ("nature") that has a "budget" $B$. This budget allows the agent to choose $B$ of the algorithm's decision variables and fix them to specific values. This defines a subclass of algorithms, allowing us to analyze the performance spectrum.

### 1. Define the "Reward" Objective

We use the original K-secretary LP objective (a maximization problem) directly.

Let $I$ be the set of all variable indices $v = (j, k, i)$.
Let $z = \{z_v\}_{v \in I}$ be the vector of all decision variables $z_{j|k}(i)$.
Let $c_v$ be the objective coefficient for variable $z_v$.

The "reward" is the original LP objective:
$$\text{Obj}(z) = \sum_{v \in I} c_v z_v$$

Let $\text{LP-Constraints}(z)$ represent the full set of original LP constraints.

### 2. Define the Performance Function $Q(y, \zeta)$

We introduce two vectors chosen by "nature":
1.  A binary vector $y = \{y_v\}_{v \in I}$, where $y_v=1$ if variable $v$ is fixed, and $y_v=0$ if it is free.
2.  A real-valued vector $\zeta = \{\zeta_v\}_{v \in I}$, which specifies the values nature assigns to the fixed variables.

The function $Q(y, \zeta)$ quantifies the *algorithm's best possible performance* (maximum reward) given nature's choices $y$ and $\zeta$:

$$Q(y, \zeta) = \max_{z} \quad \text{Obj}(z)$$
$$\text{s.t.} \quad \text{LP-Constraints}(z)$$
$$y_v (z_v - \zeta_v) = 0, \quad \forall v \in I$$

The constraint $y_v (z_v - \zeta_v) = 0$ enforces the logic:
-   If $y_v = 1$ (variable is fixed), then $z_v - \zeta_v = 0$, so $z_v = \zeta_v$.
-   If $y_v = 0$ (variable is free), then $0 = 0$, and $z_v$ is determined by the $\max_z$ optimization.

### 3. Define Best- and Worst-Case Performance

With a fixed budget $B$ (i.e., $\sum_{v \in I} y_v = B$), we can find the "best" and "worst" algorithm subclasses that "nature" can create:

$$V_{\text{best}}(B) = \max_{y: \sum y_v = B, \zeta} Q(y, \zeta)$$
*(The highest possible reward. This is the optimistic case where "nature" chooses the $B$ variables $y$ and their values $\zeta$ in the *least* harmful way.)*

$$V_{\text{worst}}(B) = \min_{y: \sum y_v = B, \zeta} Q(y, \zeta)$$
*(The lowest possible reward. This is the pessimistic, worst-case scenario where "nature" chooses the $B$ variables $y$ and their values $\zeta$ in the *most* harmful way.)*

### 4. Analyze the Full Spectrum with Risk Parameter $\alpha$

For a given risk-tolerance $\alpha \in [0, 1]$, we want to find an algorithm (i.e., a set of fixed variables $y, \zeta$) that achieves the best possible performance, *given* that this performance must be at least as good as the $\alpha$-weighted threshold:

$$\max_{y, \zeta} \quad Q(y, \zeta)$$
$$\text{s.t.} \quad \sum_{v \in I} y_v = B$$
$$Q(y, \zeta) \ge (1-\alpha)V_{\text{best}}(B) + \alpha V_{\text{worst}}(B)$$

In [ ]:
# def solve_v_worst_bilevel(n, K, B):
#     """
#     Solves the V_worst(B) bilevel problem using Pyomo and dualization.
#     """
#     print(f"\nBuilding V_worst({B}) bilevel MIP for n={n}, K={K}...")

#     # Create the outer model
#     m = pyo.ConcreteModel()
#     M_val = 1.05

#     # --- Indices (for outer vars) ---
#     m.J = pyo.Set(initialize=range(K))
#     m.K = pyo.Set(initialize=range(K))
#     m.I = pyo.Set(initialize=range(n))
#     m.VARS = m.J * m.K * m.I

#     # --- Upper Level (Nature's) Variables ---
#     m.y = pyo.Var(m.VARS, domain=pyo.Binary)
#     m.zeta = pyo.Var(m.VARS, bounds=(0.0, 1.0))
    
#     # --- Upper Level (Nature's) Constraints ---
#     m.budget = pyo.Constraint(expr=sum(m.y[v] for v in m.VARS) == B)

#     # --- Inner Level (Primal) Problem ---
#     # Get the base LP from our builder function
#     m.primal_model = create_k_secretary_lp_model(n, K, verbose=False)
    
#     # --- Add Linking Constraints ---
#     m.primal_model.linking_constrs_upper = pyo.Constraint(
#         m.VARS,
#         rule=lambda model, j, k, i: 
#             model.z[(j,k,i)] - m.zeta[(j,k,i)] <= M_val * (1.0 - m.y[(j,k,i)])
#     )

#     m.primal_model.linking_constrs_lower = pyo.Constraint(
#         m.VARS, 
#         rule=lambda model, j, k, i: 
#             model.z[(j,k,i)] - m.zeta[(j,k,i)] >= -M_val * (1.0 - m.y[(j,k,i)])
#     )

#     # --- Apply Dual Transformation ---
#     # This transforms the inner `m.primal_model` block into its dual
#     # and parameterizes it with respect to the outer `m.y` variables.
#     print("Applying 'core.lp_dual' transformation...")
#     dual_transformer = pyo.TransformationFactory('core.lp_dual')
#     dualblk = dual_transformer.create_using(m.primal_model, parameterize_wrt=[m.y, m.zeta])
#     m.add_component("worst_sensor_values_dual", dualblk)

#     # Deactivate the primal block, we now solve the MIP
#     m.primal_model.deactivate()

#     # --- Solve the Bilevel MIP ---
#     print("MIP built. Solving with Gurobi...")
#     opt = SolverFactory('gurobi')
#     opt.options['TimeLimit'] = 300
#     opt.options['MIPGap'] = 0.01
#     # opt.options['MIPFocus'] = 3
#     # opt.options['ScaleFlag'] = 2
#     results = opt.solve(m, tee=True)

#     # --- Process Results ---
#     v_worst = None
#     worst_vars = []
    
#     if results.solver.termination_condition == pyo.TerminationCondition.optimal:
#         v_worst = results.problem.lower_bound
#         print(f"\n{'='*60}\nSOLVE COMPLETE\n{'='*60}")
#         print(f"Solver Status: {results.solver.termination_condition}")
#         print(f"V_worst({B}) = {v_worst:.6f}")
        
#         print(f"\nWorst {B} variable(s) to fix to 0 (y_v = 1):")
#         for v in m.VARS:
#             # Need to use .stale=True because we're checking var values
#             # from a problem that was transformed.
#             if pyo.value(m.y[v], exception=False) > 0.5:
#                 worst_vars.append(v)
#         print(worst_vars)
        
#     else:
#         print(f"MIP optimization failed with status {results.solver.termination_condition}")

#     return v_worst, worst_vars

# solve_v_worst_bilevel(n_demo, K_demo, 1)


Building V_worst(1) bilevel MIP for n=100, K=2...
Applying 'core.lp_dual' transformation...
MIP built. Solving with Gurobi...
Read LP format model from file /var/folders/z0/594mwlc93jv68hfyqzg2rzn40000gn/T/tmpev4177np.pyomo.lp
Reading time = 0.04 seconds
x1: 401 rows, 1996 columns, 60996 nonzeros
Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.01
Set parameter MIPFocus to value 3
Set parameter ScaleFlag to value 2
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[rosetta2] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  300
MIPGap  0.01
ScaleFlag  2
MIPFocus  3

Academic license 2418152 - for non-commercial use only - registered to ml___@rice.edu
Optimize a model with 401 rows, 1996 columns and 60996 nonzeros
Model fingerprint: 0xd4af64ab
Model has 1600 quadratic objective terms
Variable types: 1596 continuous, 400 integer (400 binary)
Coefficient stat

(None, [])

In [11]:
# First, let's verify that restricting to meaningful variables doesn't change the base LP
# We'll solve the base LP with only meaningful variables and compare to original

def test_meaningful_vars_restriction(n, K, coeff_threshold=1e-3):
    """
    Test if restricting to meaningful variables changes the optimal objective.
    """
    print(f"\n{'='*60}")
    print(f"TESTING: Does restricting to meaningful vars change optimal value?")
    print(f"{'='*60}\n")
    
    # Compute meaningful variables
    delta = np.zeros((n, K, K))
    for i in range(1, n + 1):
        for k in range(1, K + 1):
            for ell in range(k, K + 1):
                delta[i-1, k-1, ell-1] = compute_delta(n, i, k, ell)
    
    c_dict = {}
    for j in range(K):
        for k in range(K):
            for i in range(n):
                c_val = 0.0
                for ell in range(k, K):
                    c_val += (1.0 / n) * delta[i, k, ell]
                c_dict[(j, k, i)] = c_val
    
    meaningful_vars = []
    for j in range(K):
        for k in range(K):
            for i in range(n):
                if i < k or i == 0:
                    continue
                if abs(c_dict[(j, k, i)]) > coeff_threshold:
                    meaningful_vars.append((j, k, i))
    
    print(f"Meaningful variables: {len(meaningful_vars)} / {K*K*n}")
    
    # Solve base LP (unrestricted)
    base_model, obj_unrestricted = solve_base_lp_pyomo(n, K, verbose=False)
    print(f"\nUnrestricted LP objective: {obj_unrestricted:.6f}")
    
    # Now solve with non-meaningful variables constrained to 0
    model_restricted = create_k_secretary_lp_model(n, K, verbose=False)
    
    # Constrain non-meaningful vars to 0
    for j in range(K):
        for k in range(K):
            for i in range(n):
                if (j, k, i) not in meaningful_vars:
                    model_restricted.z[(j, k, i)].fix(0.0)
    
    opt = SolverFactory('gurobi')
    results = opt.solve(model_restricted, tee=False)
    
    if results.solver.termination_condition == pyo.TerminationCondition.optimal:
        obj_restricted = pyo.value(model_restricted.objective)
        print(f"Restricted LP objective:   {obj_restricted:.6f}")
        print(f"Difference:                {abs(obj_unrestricted - obj_restricted):.10f}")
        
        if abs(obj_unrestricted - obj_restricted) < 1e-6:
            print(f"\n✓ PASS: Restriction does not change optimal value!")
            return True
        else:
            print(f"\n✗ FAIL: Restriction changes optimal value by {abs(obj_unrestricted - obj_restricted):.6f}")
            return False
    else:
        print(f"Restricted LP failed to solve")
        return False

# Run the test
test_meaningful_vars_restriction(n_demo, K_demo)


TESTING: Does restricting to meaningful vars change optimal value?

Meaningful variables: 378 / 400

Unrestricted LP objective: 0.987887
Restricted LP objective:   0.987887
Difference:                0.0000000000

✓ PASS: Restriction does not change optimal value!


True

## Computing V_best and V_worst

For a fixed budget $B$:
- $V_{\text{best}}(B) = V_{\text{best}}(0)$ = optimal LP value (nature cannot improve beyond optimal)
- $V_{\text{worst}}(B)$ = minimum performance when nature fixes $B$ variables to 0

We use **enumeration** to compute $V_{\text{worst}}(B)$ exactly.

In [8]:
# Helper function to identify meaningful variables
def get_meaningful_vars(n, K, coeff_threshold=1e-3):
    """Identify variables with non-negligible objective coefficients."""
    delta = np.zeros((n, K, K))
    for i in range(1, n + 1):
        for k in range(1, K + 1):
            for ell in range(k, K + 1):
                delta[i-1, k-1, ell-1] = compute_delta(n, i, k, ell)
    
    c_dict = {}
    for j in range(K):
        for k in range(K):
            for i in range(n):
                c_val = 0.0
                for ell in range(k, K):
                    c_val += (1.0 / n) * delta[i, k, ell]
                c_dict[(j, k, i)] = c_val
    
    meaningful_vars = []
    for j in range(K):
        for k in range(K):
            for i in range(n):
                if i < k or i == 0:
                    continue
                if abs(c_dict[(j, k, i)]) > coeff_threshold:
                    meaningful_vars.append((j, k, i))
    
    return meaningful_vars, c_dict


def create_k_secretary_lp_model_restricted(n, K, meaningful_vars, verbose=False):
    """Creates LP with only meaningful variables (non-meaningful implicitly 0)."""
    delta = np.zeros((n, K, K))
    for i in range(1, n + 1):
        for k in range(1, K + 1):
            for ell in range(k, K + 1):
                delta[i-1, k-1, ell-1] = compute_delta(n, i, k, ell)
    
    model = pyo.ConcreteModel()
    model.VARS = pyo.Set(initialize=meaningful_vars)
    model.z = pyo.Var(model.VARS, bounds=(0.0, 1.0))

    # Objective
    c_dict = {}
    for j, k, i in model.VARS:
        c_val = 0.0
        for ell in range(k, K):
            c_val += (1.0 / n) * delta[i, k, ell]
        c_dict[(j,k,i)] = c_val
    
    model.objective = pyo.Objective(
        expr=sum(c_dict[v] * model.z[v] for v in model.VARS),
        sense=pyo.maximize
    )

    # Constraints (only for meaningful variables)
    model.lp_constrs = pyo.ConstraintList()

    # Constraints for j < K
    for j in range(K - 1):
        for k in range(K):
            for i in range(1, n):
                if (j, k, i) not in meaningful_vars:
                    continue
                
                rhs = 0
                for m in range(i):
                    for ell in range(K):
                        if (j+1, ell, m) in meaningful_vars:
                            rhs += (1.0 / (m + 1)) * model.z[(j+1, ell, m)]
                        if (j, ell, m) in meaningful_vars:
                            rhs -= (1.0 / (m + 1)) * model.z[(j, ell, m)]
                
                model.lp_constrs.add(model.z[(j, k, i)] <= rhs)
    
    # Constraints for j = K
    j = K - 1
    for k in range(K):
        for i in range(1, n):
            if (j, k, i) not in meaningful_vars:
                continue
            
            rhs = 1.0
            for m in range(i):
                for ell in range(K):
                    if (j, ell, m) in meaningful_vars:
                        rhs -= (1.0 / (m + 1)) * model.z[(j, ell, m)]
            
            model.lp_constrs.add(model.z[(j, k, i)] <= rhs)
        
    return model


def solve_v_worst_enumeration(n, K, B, coeff_threshold=1e-3):
    """
    Solve V_worst(B) by enumerating all C(|meaningful_vars|, B) combinations.
    Nature fixes B variables to 0 (the worst case).
    
    Returns the minimum objective value and which variables to fix to 0.
    """
    meaningful_vars, c_dict = get_meaningful_vars(n, K, coeff_threshold)
    
    print(f"Solving V_worst({B}) by enumeration...")
    print(f"  Meaningful variables: {len(meaningful_vars)}")
    print(f"  Combinations to try: {comb(len(meaningful_vars), B, exact=True)}")
    
    from itertools import combinations
    
    v_worst = float('inf')
    worst_vars = None
    opt = SolverFactory('gurobi')
    
    # Try all combinations of B variables to fix to 0
    for i, vars_to_fix in enumerate(combinations(meaningful_vars, B)):
        if i % 100 == 0:
            print(f"  Progress: {i}/{comb(len(meaningful_vars), B, exact=True)}", end='\r')
        
        model = create_k_secretary_lp_model_restricted(n, K, meaningful_vars)
        
        # Fix selected variables to 0
        for v in vars_to_fix:
            model.z[v].fix(0.0)
        
        results = opt.solve(model, tee=False)
        
        if results.solver.termination_condition == pyo.TerminationCondition.optimal:
            obj_val = pyo.value(model.objective)
            if obj_val < v_worst:
                v_worst = obj_val
                worst_vars = vars_to_fix
        
        for v in vars_to_fix:
            model.z[v].unfix()
    
    print(f"\n  V_worst({B}) = {v_worst:.6f}")
    
    return v_worst, list(worst_vars) if worst_vars else []


# Demo: Compute V_best and V_worst for B=1
B_demo = 1
print(f"\n{'='*60}\nComputing V_best and V_worst for B={B_demo}\n{'='*60}\n")

v_best = obj_val_base  # Optimal LP value
v_worst, worst_vars = solve_v_worst_enumeration(n_demo, K_demo, B_demo)

print(f"\n{'='*60}")
print(f"V_best({B_demo})  = {v_best:.6f}  (competitive ratio: {v_best/K_demo:.6f})")
print(f"V_worst({B_demo}) = {v_worst:.6f}  (competitive ratio: {v_worst/K_demo:.6f})")
if worst_vars:
    print(f"\nWorst-case configuration (variables fixed to 0):")
    for v in worst_vars:
        j, k, i = v
        print(f"  z[{j},{k},{i}] = 0")
print(f"{'='*60}")


Computing V_best and V_worst for B=1

Solving V_worst(1) by enumeration...
  Meaningful variables: 378
  Combinations to try: 378
  Progress: 300/378
  V_worst(1) = 0.983489

V_best(1)  = 0.987887  (competitive ratio: 0.493943)
V_worst(1) = 0.983489  (competitive ratio: 0.491744)

Worst-case configuration (variables fixed to 0):
  z[1,0,42] = 0


## Finding Algorithms with Risk Parameter $\alpha$

Given $\alpha \in [0,1]$, find the **best** algorithm subject to a performance guarantee:

$$\max_{y, \zeta} Q(y, \zeta)$$
$$\text{s.t.} \quad \sum_v y_v = B$$
$$Q(y, \zeta) \leq (1-\alpha) V_{\text{best}}(B) + \alpha V_{\text{worst}}(B)$$

where:
- $\alpha = 0$: Find best algorithm (may be risky)
- $\alpha = 1$: Find algorithm matching worst-case (most conservative)
- $\alpha \in (0,1)$: Trade-off between performance and robustness

In [10]:
def solve_alpha_risk_algorithm(n, K, B, alpha, v_best, v_worst, coeff_threshold=1e-3):
    """
    Find algorithm (y, zeta) that maximizes performance subject to:
        Q(y, zeta) >= (1-alpha)*v_best + alpha*v_worst
    
    Uses enumeration over y (which vars to fix) and optimization over zeta (what values).
    
    Args:
        alpha: Risk parameter in [0,1]. 0=best, 1=worst-case
        v_best: V_best(B) value
        v_worst: V_worst(B) value
    
    Returns:
        (performance, vars_to_fix, fixed_values)
    """
    meaningful_vars, c_dict = get_meaningful_vars(n, K, coeff_threshold)
    
    threshold = (1 - alpha) * v_best + alpha * v_worst
    
    print(f"\nFinding α-risk algorithm with α={alpha}, B={B}...")
    print(f"  Performance threshold: {threshold:.6f}")
    print(f"  Enumerating over {comb(len(meaningful_vars), B, exact=True)} combinations...")
    
    from itertools import combinations
    
    best_performance = -float('inf')
    best_config = None
    opt = SolverFactory('gurobi')
    
    # For each choice of which B variables to fix
    for i, vars_to_fix in enumerate(combinations(meaningful_vars, B)):
        if i % 100 == 0:
            print(f"  Progress: {i}/{comb(len(meaningful_vars), B, exact=True)}", end='\r')
        
        # For each choice of what values to fix them to
        # Since we're maximizing, we want to set them to maximize objective
        # subject to threshold constraint
        
        # Try setting to their "optimal" values (from unconstrained LP)
        model_unconstrained = create_k_secretary_lp_model_restricted(n, K, meaningful_vars)
        results = opt.solve(model_unconstrained, tee=False)
        
        if results.solver.termination_condition != pyo.TerminationCondition.optimal:
            continue
        
        # Get optimal z values for the variables we want to fix
        zeta_vals = {v: pyo.value(model_unconstrained.z[v]) for v in vars_to_fix}
        
        # Now solve LP with these variables fixed to zeta_vals
        model_fixed = create_k_secretary_lp_model_restricted(n, K, meaningful_vars)
        for v in vars_to_fix:
            model_fixed.z[v].fix(zeta_vals[v])
        
        results_fixed = opt.solve(model_fixed, tee=False)
        
        if results_fixed.solver.termination_condition == pyo.TerminationCondition.optimal:
            performance = pyo.value(model_fixed.objective)
            
            # Check if meets threshold and improves best
            if performance >= threshold and performance > best_performance:
                best_performance = performance
                best_config = (vars_to_fix, zeta_vals)
    
    print(f"\n  Best performance found: {best_performance:.6f}")
    
    if best_config:
        vars_to_fix, zeta_vals = best_config
        return best_performance, list(vars_to_fix), zeta_vals
    else:
        print("  No configuration meets the threshold!")
        return None, None, None


# Demo: Find algorithms for different risk levels
print(f"\n{'='*60}\nFinding α-risk algorithms\n{'='*60}\n")

for alpha in [0.2, 0.5]:
    perf, vars_fixed, zeta_vals = solve_alpha_risk_algorithm(
        n_demo, K_demo, B_demo, alpha, v_best, v_worst
    )
    
    if perf is not None:
        print(f"\nα={alpha}: Performance = {perf:.6f}")
        if vars_fixed:
            for v in vars_fixed:
                j, k, i = v
                print(f"    Fix z[{j},{k},{i}] = {zeta_vals[v]:.6f}")
    print()


Finding α-risk algorithms


Finding α-risk algorithm with α=0.2, B=1...
  Performance threshold: 0.987007
  Enumerating over 378 combinations...
  Progress: 300/378
  Best performance found: 0.987887

α=0.2: Performance = 0.987887
    Fix z[1,0,44] = 0.500000


Finding α-risk algorithm with α=0.5, B=1...
  Performance threshold: 0.985688
  Enumerating over 378 combinations...
  Progress: 300/378
  Best performance found: 0.987887

α=0.5: Performance = 0.987887
    Fix z[1,0,44] = 0.500000

